In [2]:
from tqdm import tqdm
import pandas as pd
from glob import glob

In [3]:
files = sorted(glob('./data/*.csv'))
data_fit = pd.DataFrame()
for file in tqdm(files[:1]):
    df_day = pd.read_csv(file, usecols=["okt"])
    data_fit = pd.concat([data_fit, df_day])
data_fit.reset_index(drop=True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.60it/s]


In [4]:
data_fit

,okt
0,한국 배구 연맹 23일 주말 열리다 프로배구 4 경기 연기 다 고 1일 밝히다 중계...
1,아동 성범죄자 조두순 69 사진 지난달 추다 뒤 보름 외부 활동 나서다 것 전해지다...
2,서울 신문 추미애 구치소 집단 감염 첫 사과 집단 감염 송구 취약 부분 드러나다 빠...
3,정부 수도권 사회 거리 두기 조 정안 2일 발표 하다 하다 상황 지금 단계 몇 주 ...
4,강릉 뉴스 1 이종재 기자 새해 첫날 1일 강원 강릉 지역 찾다 해 맞이 객 지난해...
...,...
1207,정읍 뉴시스 김얼 기자 2021년 신축년 희다 소 띠 해 상서 기운 물씬 일어나다 ...
1208,정읍 뉴시스 김얼 기자 2021년 신축년 희다 소 띠 해 상서 기운 물씬 일어나다 ...
1209,서울 뉴시스 홍세 희다 기자 최대 집 대한의사협회 1일 새해 94 의정 합의 정신 ...
1210,서울 뉴시스 구무 기자 김용익 건강 보험 공단 이사장 오다 한해 건강 보험 보장 률...


In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
model = SentenceTransformer('ddobokki/klue-roberta-small-nli-sts')

In [40]:
%%time
embeddings = model.encode(data_fit.iloc[:100,0], show_progress_bar=True)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 1min 54s
Wall time: 28.6 s


In [41]:
arc_content = """2011년 한국을 떠나 러시아로 귀화한 전 쇼트트랙 선수 빅토르 안(38·한국명 안현수)의 국내 복귀 시도가 일단 무산됐다. 경기도 성남시청 직장운동부 빙상팀 코치에 지원했으나, 최종 후보에 들지 못해서다.

성남시는 29일 “시청 빙상팀 코치직 채용 전형에 빅토르 안을 포함해 7명이 지원했는데, 빅토르 안은 상위 2배수 후보에 들지 않았다”고 밝혔다. 시 관계자는 “서류와 면접 심사를 통해 기술, 소통 능력 등 여러 요소를 종합해 판단했다”며 “빙상계 여론과 언론 보도 등을 통해 나오는 시각도 평가에 반영됐다”고 했다.

성남시는 지난해 12월 19일 빙상팀 코치를 뽑기 위한 채용 공고를 냈다. 빅토르 안과 베이징 동계올림픽에서 중국 대표팀을 이끈 김선태 전 감독 등 7명이 지원했다. 김 전 감독도 최종 후보에 들지 못한 것으로 전해졌다.

빅토르 안은 2006년 토리노 동계올림픽에서 3개의 금메달을 딴 한국 쇼트트랙의 간판이었다. 2011년 소속팀이었던 성남시청이 빙상팀을 해체하자 선수 생활을 이어가기 위해 러시아로 귀화했다. 2014년 소치 동계올림픽에는 러시아 선수로 나서 3관왕에 올랐다. 2018년 평창 동계올림픽 출전이 무산된 이후에는 은퇴를 선언하고 지도자로 변신했다. 2022년 베이징 동계올림픽에서는 중국 대표팀 코치로 활동했다.

성남시는 31일 빙상팀 코치 선발 결과를 발표할 예정이다."""

In [42]:
%%time
embeded_content = model.encode([arc_content])

CPU times: total: 1.06 s
Wall time: 272 ms


In [52]:
embeded_content[0][:10]

array([ 0.05949196,  0.5816361 ,  0.16172582, -0.09110567, -0.30458847,
        0.20078555, -0.10215177,  0.22305581, -0.29024214, -0.23665504],
      dtype=float32)

In [26]:
from sentence_transformers import util
import torch

In [63]:
%%time
cos_scores = util.pytorch_cos_sim(embeded_content, embeddings)[0]

CPU times: total: 0 ns
Wall time: 1.28 ms


In [88]:
top_k = 5
top_res = torch.topk(cos_scores, k=top_k)
for i, (score, idx) in enumerate(zip(top_res[0], top_res[1])):
    print(f"{i+1}: {data_fit.iloc[idx.tolist()]} {'(유사도: {:.4f})'.format(score)}\n")

1: okt    원주 뉴시스 장경일 기자 강원 원주시 평창군 1일 오후 2 명의 신종 코로나바이러스...
Name: 18, dtype: object (유사도: 0.3015)

2: okt    서울 뉴스 1 이형진 기자 이영 기자 방역 당국 2일 사회 거리 두기 단계 조 정안...
Name: 83, dtype: object (유사도: 0.2980)

3: okt    서울 뉴시스 심 동준 류 인선 기자 서울 영등포구 트윈타워 청소 노동 하다 용역 업...
Name: 81, dtype: object (유사도: 0.2834)

4: okt    원주 뉴시스 장경일 기자 강원도 새해 첫날 1일 하루 동안 25 명의 신종 코로나바...
Name: 88, dtype: object (유사도: 0.2799)

5: okt    강원 뉴스 1 이종재 기자 1일 강원 지역 코로나 19 신규 확 진자 26 명 발생...
Name: 98, dtype: object (유사도: 0.2761)



In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [90]:
tfidf_vec = TfidfVectorizer()

In [92]:
%%time
tfidf_dtm = tfidf_vec.fit_transform(data_fit.iloc[:100,0])

CPU times: total: 31.2 ms
Wall time: 54 ms


In [93]:
%%time
tfidf_content = tfidf_vec.transform([arc_content])

CPU times: total: 0 ns
Wall time: 1.03 ms


In [94]:
from sklearn.metrics.pairwise import cosine_similarity

In [95]:
%%time
cos_sim_res2 = cosine_similarity(tfidf_content, tfidf_dtm)

CPU times: total: 0 ns
Wall time: 4.09 ms
